# LAB 01 Topic models - Est Republicain - Gensim - LDA vs LSA

Dans ce premier lab nous allons explorer un corpus en français propre et bien structuré de façon à nous familiariser avec les outils de topic models.

Nous travaillerons avec la librairie:

* [Gensim](https://radimrehurek.com/gensim/) en python

Nous verrons comment numériser le texte original brut afn de le rendre compatible avec une analyse avec Gensim. 

Le but étant de construire la matrice mots-documents



# Le corpus 

Constitué des articles publiés dans l'Est Républicain de mai à septembre 1999. 

Le corpus original se trouve sur http://www.cnrtl.fr/corpus/estrepublicain/. 

133 fichiers au format XML a partir desquels ont été extrait 30.000 paragraphes.

Le corpus sur lequel nous allons travailler est sur ```../data/estrepublicain_annee_1999.csv```. 



# 1. Chargement des données

* Charger le document dans une dataframe Panda



In [1]:
import pandas as pd

DATA_PATH = '../data/'
filename  = 'estrepublicain_annee_1999.csv'

df = pd.read_csv(DATA_PATH + filename)

print("\nLe corpus contient {} rows et {} colonne(s)".format(df.shape[0], df.shape[1]))

print("\nColonnes: {} ".format(df.columns))

print("\n== Premier elements: ")

print(df.head().values)

# Ne garder que les 1000 premiers commentaires

df = df[0:1000]
    


Le corpus contient 30241 rows et 1 colonne(s)

Colonnes: Index(['text'], dtype='object') 

== Premier elements: 
[[ "André Bauer, le Bonhomme de St-Dié ; Alain Dagosto, Bières de Vézelize (54) ; Paulette Gay, Pieds de cochons de Dommartin ; Corinne Dexemple, Tourte néocastrienne ; Bernadette Paulin, Pâté lorrain de Châtenois ; Andrée Labrux, Pétrou de Senones ; Marie-Thérèse Muller, Cochonneux de la Seille de Sillegny (57) ; Ginette Laporte, Eau de Contrexéville ; Danièle Richard, Framboise saulxuronne ; Josette Pouchucq, Cuisses de grenouilles de Vittel ; Daniel Léonard, Nostre damme de Chiney (Belgique) ; Sophie Valdenaire, Pissenlit de Xertigny ; Simone Leick, Cochon d'autrefois de Sierck les Bains (57) ; Yves Lievens, Kuulkappers de St-Gilles (Belgique) ; Brigitte Lievens, Miel de montagne de Plombières ; Louise Fallot, Madeleine de Commercy ; Claude Himbert, Macaron et bergamote de Nancy ; Robert Fuchs, Marmite d'or ; Jean-Pierre Ruspini, Saumon de Salm ; Jean-Marie Mougel, Andou

# 2. Transformation avant numerisation

Pour appliquer les models LDA et LSI de Gensim on doit d'abord construire la matrice mots - documents.
Ce qui implique de tokenizer le contenu.

## Tokenization

_converting a sequence of characters into a sequence of tokens_

Dans le context de l'analyse textuelle, un token peut etre 

* une syllabe,
* un mot
* un bigram ou n-gram, un skip-gram, ....

Pour extraire les tokens d'une phrase, on peut simplement splitter sur l'espace:

``` 'les médiateurs confirment « l'absence de solution parfaite ». '.split(' ') ```

Ce qui donne

``` ['les', 'médiateurs', 'confirment', '«', "l'absence, 'de', 'solution', 'parfaite', '».', ''] ```

Noter l'espace à la fin, et les tokens de ponctuation

On peut aussi utiliser NLTK.


In [2]:
from nltk import word_tokenize

sentence = "les médiateurs confirment « l'absence de solution parfaite ». "

tokens = word_tokenize(sentence)
print(tokens)

['les', 'médiateurs', 'confirment', '«', "l'absence", 'de', 'solution', 'parfaite', '»', '.']


=> Séparation de ```'»', '.'```


## Ponctuation. 

Enlever les caractères de ponctuation des tokens.

On utilise pour cela la librairie ```string``` et on crée un ```translator``` a partir de la liste des caracteres à supprimer.

* Lister les caracteres à éviter
```
punctuation_chars = ''.join([s for s in string.punctuation if s not in ['_',"'",'-']  ]) + '\r\n“”…»«' + "’"
```

* Transformer en dictionnaire 
```
{ 
    ';': ' ',    # point virgule transformé en espace
    '.': ' ',    # point  transformé en espace
    ect ...
}
```

In [3]:
import string

print("Liste original de signe de ponctuations")
print("\t{}".format(string.punctuation))


sentence = "les médiateurs confirment « l'absence de solution parfaite ». "

punctuation_chars = ''.join([s for s in string.punctuation if s not in ['_', '-']  ]) + '“”…»«’\r\n'
print("Liste étendue de signe de ponctuations")
print("\t{}".format(punctuation_chars))


#  Construction d'un dict { '~':' ', '$': ' ', ... }
d = {}
for k in punctuation_chars:
    d[k] = ' '

# L'operateur de translation
translator = str.maketrans(d)

sentence = "les médiateurs confirment « l'absence de solution parfaite ». Et ils n'en “pensent pas moins”!"
new_sentence = sentence.translate(translator)

print()
print("=== phrase originale \n\t{}\n".format(sentence))
print("=== sans la ponctuation \n\t{}\n".format(new_sentence))
                                  
                                  

Liste original de signe de ponctuations
	!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
Liste étendue de signe de ponctuations
	!"#$%&'()*+,./:;<=>?@[\]^`{|}~“”…»«’


=== phrase originale 
	les médiateurs confirment « l'absence de solution parfaite ». Et ils n'en “pensent pas moins”!

=== sans la ponctuation 
	les médiateurs confirment   l absence de solution parfaite    Et ils n en  pensent pas moins  



## Stopwords

Liste de mots très fréquents, qui n'apportent rien en terme d'information


In [ ]:
from nltk.corpus import stopwords

print("=== stopwords - français: \n{}".format(stopwords.words('french')))

## Nettoyage  du corpus


* Enlever la ponctuation des paragraphes
* tokeniser


On travail maintenant sur la DataFrame avec le pattern

``` df['new_column']   = df.text.apply( lambda d : fonction(d)    ) ```

où new_column est la nouvelle representation des textes,  et fonction est la fonction que nous voulons appliquer: lower, stopwords, ponctuation, ...

Dans l'appel a ```lambda```,  ```d``` est un row de la DataFrame ```df.text```

Le code ci dessus est equivalent à 

```
    for i, d in df.iterrows():
          df.lock[i,'new_column'] = fonction(d)
```

Mais la pattern ```df apply lambda``` est bien plus rapide

## Enlever la ponctuation 

On applique le translator definit auparavant


In [ ]:
df['text_no_punctuation'] = df.text.apply(lambda d : ( d.translate(translator) ) )


# df.loc[5].text: le text du 5ieme row

print("\n=== Avant: \n\t{}".format(df.loc[5].text))
print("\n=== Apres: \n\t{}".format(df.loc[5].text_no_punctuation))


# Tokenization

On applique la fonction ```word_tokenize``` 


In [ ]:
# Tokenization et minuscule
df['tokens_all']  = df.text_no_punctuation.apply(lambda s : word_tokenize(s.lower()))

# Stopwords

On part d'une liste predéfinie que l'on complete au fur et a mesure.

In [ ]:
# Stopwords

list_stopwords = stopwords.words('french')

# a partir d'une liste de tokens, retourne une liste sans les stopwords
def remove_stopword(tokens):
     return [w for w in tokens if (w not in list_stopwords) ]

df['tokens'] = df.tokens_all.apply(lambda tks : remove_stopword(tks) )

print("\n=== Original: \n\t{}".format(df.loc[6].text_no_punctuation))
print("\n=== Tokens: \n\t{}".format(df.loc[6].tokens_all))
print("\n=== Sans stopwords: \n\t{}".format(df.loc[6].tokens))




## Réduire la dataframe

Enlever les variables intermediaires


In [ ]:
print("df a maintenant {} rows et {} colonnes: \n{}".format( df.shape[0], df.shape[1], df.columns ))


print("\n* Ne garder que les colonnes text et tokens ")

df = df[['text', 'tokens']]

print("\n* Calcul du nombre de tokens")

df['len_tokens'] = df.tokens.apply(len)

print("\n* Distribution de la longueur des paragraphes tokenizés")

print(df.describe())


### Ne garder que les longs paragraphes

Exemple de subsetting une dataframe avec une condition sur une colonne

```
condition = df.len_token > 20
df = df[condition] ```

où ```condition``` est une serie de booleen ```[True, False, True, True, .... , False]```


In [ ]:
df = df[ df.len_tokens > 60 ]

print(df.shape)

## Numerisation  
Encore un effort

Nous sommes presque à l'etape du LDA.


#### Dictionnaire

Il faut d'abord construire le dictionnaire global

```dictionary  = corpora.Dictionary(df.tokens)```


Dictionary(3884 unique tokens: ['andré', 'bauer', 'bonhomme', 'st-dié', 'alain']...)

#### Corpus

Il faut ensuite construire la matrice mots - documents, numeriser les documents grace au dictionary

```df['corpus'] = df.tokens.apply(lambda d : dictionary.doc2bow(d))```

on obtient:
0    [(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1) ...]
1    [(61, 3), (122, 1), (123, 1), (124, 3), (125, ...]

Et ça ce lit de la façcon suivante:

Dans le document 1, le mot 61 apparait 3 fois, le mot 122: 1 fois, ...


In [ ]:
from gensim import corpora, models

# Dictionnaire de tous les tokens
dictionary  = corpora.Dictionary(df.tokens)
print(dictionary)

In [ ]:
df['corpus'] = df.tokens.apply(lambda d : dictionary.doc2bow(d))

print(df['corpus'] .head())

# aggreger les rows

corpus = [c for c in df.corpus ]


###  LDA

* Definir le nombre de topic
* Appliquer ```models.LdaModel```
* Print les resultats




In [ ]:
# Nombre de topics: le parametre principal

num_topics= 8

# Le model LDA
lda = models.LdaModel(corpus,
    id2word=dictionary,
    num_topics=num_topics,
    alpha = 'asymmetric',
    eta='auto',
    passes = 2,
    iterations = 20
)


for t in lda.show_topics(num_topics=num_topics, formatted=True, log = False):
    print(t)


# Autre Technique: LSI


Necessite la TF-IDF matrice. Version normalisée de la matrice words - documents

```tfidf = models.TfidfModel(corpus)```


```corpus_tfidf = tfidf[corpus]```

```lsi = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=2)```






In [ ]:
tfidf = models.TfidfModel(corpus)


corpus_tfidf = tfidf[corpus]

for doc in corpus_tfidf[0:3]: print(doc)

num_topics= 8



In [ ]:
# LSA

lsi = models.LsiModel(corpus_tfidf , id2word=dictionary, num_topics=num_topics)
for t in lsi.show_topics(num_topics=num_topics, formatted=True, log = False):
    print(t)
    
    

# A votre tour

En utilisant maintenant tout le corpus à disposition, essayez d'extraire les meilleurs topics 

* Faites varier le numbre de topics
* Ajoutez des stopwords à la liste
* Comparez LDA vs LSI. Qu'est ce qui marche le mieux?


